In [1]:
# store start time to get execution time of entire script
import time
start_time = time.time()

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import csv

df = pd.read_csv('../data/data_cleaned.csv')
df.head()

,Unnamed: 0,ID,State,City,agyaddr,xobsyr_0,Illicit_Days,Illicit_Cens,female,nonwhite,unemplmt_cd,prsatx_cd,gvsg_cd,CWSg_0_cd,dssg_0_cd,epsg_0_cd,adhdg_0_cd,cdsg_0_cd,cjsig_0_cd,lrig_0_cd,srig_0_cd,SESg_0_cd,r4ag_0_cd,SUDSy_0_cd,homeless_0_cd,ncar_cd,TRIg_0_cd,PYS9Sxg_cd,primsev_other,primsev_alcohol,primsev_amphetamines,primsev_cocaine,primsev_marijuana,primsev_opioids,B2a_0g
0,0,13907,WV,Huntington,3375 U.S. RT. 60 East,2007,192,0,0,0,1,0,2,0,1,1,2,1,0,1,1,1,2,2,0,1,0,0,0,1,0,0,0,0,0
1,2,4271,NJ,Red Bank,270 Highway 35,2005,43,1,0,0,0,0,2,1,0,0,0,0,0,1,2,2,1,2,0,0,1,0,0,0,0,0,1,0,1
2,4,2280,IL,Bloomington,720 W Chestnut St,2003,365,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,1,1,0,0,0,0,0,1,0,0,0,0,1
3,5,285,MO,St. Louis,5400 Arsenal St.,2004,362,0,0,1,0,0,2,0,1,1,2,2,1,1,1,0,2,2,0,0,0,0,0,0,0,0,1,0,0
4,6,21058,ID,Boise,8620 W. Emerald Street,2011,365,0,0,1,0,1,2,0,2,1,2,1,2,2,2,1,1,3,1,1,0,3,0,0,1,0,0,0,2


In [3]:
df['Address'] = df['agyaddr'] + ', ' + df['City'] + ', ' + df['State']
addresses = list(df['Address'].unique())
len(addresses)

188

In [4]:
%%time

from opencage.geocoder import OpenCageGeocode
from tqdm.notebook import tqdm

# get longitude and latitude for each unique address
key = '2d4ad4cfcdc44021abfe1e347daa6dc7'
geocoder = OpenCageGeocode(key)

geocoded_addrs = []

for addr in tqdm(addresses):
    results = geocoder.geocode(addr)
    lat = results[0]['geometry']['lat']
    lng = results[0]['geometry']['lng']
    geocoded_addrs.append([addr, lat, lng])
    # print(addr + ' -> (' + str(lat) + ', ' + str(lon) + ')')


CPU times: user 4.44 s, sys: 520 ms, total: 4.96 s
Wall time: 3min


In [5]:
geocoded_addrs_df = pd.DataFrame(geocoded_addrs, columns=['Address', 'lat', 'lng'])
geocoded_addrs_df.head()

,Address,lat,lng
0,"3375 U.S. RT. 60 East , H...",38.402682,-82.505588
1,"270 Highway 35 , R...",40.365628,-74.083096
2,"720 W Chestnut St , B...",40.485418,-89.000689
3,"5400 Arsenal St. , S...",38.605999,-90.280051
4,"8620 W. Emerald Street , B...",43.611373,-116.288882


In [6]:
geocoded_addrs_df.isna().sum()

Address    0
lat        0
lng        0
dtype: int64

In [7]:
%%time

import urllib, json, requests
import numpy as np

# get block_fips code based on longitude and latitude
def api_call(row):
    lat = row['lat']
    # http://data.fcc.gov/api/block/find?format=json&latitude=28.35975&longitude=-81.421988&showall=true
    lng = row['lng']
    URL = "https://geo.fcc.gov/api/census/block/find?format=json&latitude=" + str(lat) + "&longitude=" + str(lng)
    with urllib.request.urlopen(URL) as url:
        data = json.loads(url.read().decode())
    try:
        return int(data['County']['FIPS']), int(data['Block']['FIPS']), data['State']['name']
    except TypeError: # response failed to get county fips
        return np.nan, np.nan, np.nan

geocoded_addrs_df['state_name'] = ''
for ind,row in tqdm(geocoded_addrs_df.iterrows(), total=geocoded_addrs_df.shape[0]):
    county_FIPS, block_FIPS, state_name = api_call(row)
    geocoded_addrs_df.set_value(ind, 'county_FIPS', county_FIPS)
    geocoded_addrs_df.set_value(ind, 'block_FIPS', block_FIPS)
    geocoded_addrs_df.set_value(ind, 'state_name', state_name)
    
geocoded_addrs_df.head()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead



CPU times: user 3.24 s, sys: 280 ms, total: 3.52 s
Wall time: 1min 21s


,Address,lat,lng,state_name,county_FIPS,block_FIPS
0,"3375 U.S. RT. 60 East , H...",38.402682,-82.505588,West Virginia,54099.0,5.409901e+14
1,"270 Highway 35 , R...",40.365628,-74.083096,New Jersey,34025.0,3.402580e+14
2,"720 W Chestnut St , B...",40.485418,-89.000689,Illinois,17113.0,1.711300e+14
3,"5400 Arsenal St. , S...",38.605999,-90.280051,Missouri,29510.0,2.951013e+14
4,"8620 W. Emerald Street , B...",43.611373,-116.288882,Idaho,16001.0,1.600100e+14


In [8]:
geocoded_addrs_df.isna().sum() # how many nulls per column

Address        0
lat            0
lng            0
state_name     1
county_FIPS    1
block_FIPS     1
dtype: int64

In [9]:
# re-merge patient data and geocoded addresses by the address feature
df = pd.merge(df, geocoded_addrs_df, on=['Address'], how='left')

In [10]:
df = df.dropna() # remove any remaining null-valued rows

In [11]:
df.shape

(21896, 41)

In [12]:
df.head()

,Unnamed: 0,ID,State,City,agyaddr,xobsyr_0,Illicit_Days,Illicit_Cens,female,nonwhite,unemplmt_cd,prsatx_cd,gvsg_cd,CWSg_0_cd,dssg_0_cd,epsg_0_cd,adhdg_0_cd,cdsg_0_cd,cjsig_0_cd,lrig_0_cd,srig_0_cd,SESg_0_cd,r4ag_0_cd,SUDSy_0_cd,homeless_0_cd,ncar_cd,TRIg_0_cd,PYS9Sxg_cd,primsev_other,primsev_alcohol,primsev_amphetamines,primsev_cocaine,primsev_marijuana,primsev_opioids,B2a_0g,Address,lat,lng,state_name,county_FIPS,block_FIPS
0,0,13907,WV,Huntington,3375 U.S. RT. 60 East,2007,192,0,0,0,1,0,2,0,1,1,2,1,0,1,1,1,2,2,0,1,0,0,0,1,0,0,0,0,0,"3375 U.S. RT. 60 East , H...",38.402682,-82.505588,West Virginia,54099.0,5.409901e+14
1,2,4271,NJ,Red Bank,270 Highway 35,2005,43,1,0,0,0,0,2,1,0,0,0,0,0,1,2,2,1,2,0,0,1,0,0,0,0,0,1,0,1,"270 Highway 35 , R...",40.365628,-74.083096,New Jersey,34025.0,3.402580e+14
2,4,2280,IL,Bloomington,720 W Chestnut St,2003,365,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,1,1,0,0,0,0,0,1,0,0,0,0,1,"720 W Chestnut St , B...",40.485418,-89.000689,Illinois,17113.0,1.711300e+14
3,5,285,MO,St. Louis,5400 Arsenal St.,2004,362,0,0,1,0,0,2,0,1,1,2,2,1,1,1,0,2,2,0,0,0,0,0,0,0,0,1,0,0,"5400 Arsenal St. , S...",38.605999,-90.280051,Missouri,29510.0,2.951013e+14
4,6,21058,ID,Boise,8620 W. Emerald Street,2011,365,0,0,1,0,1,2,0,2,1,2,1,2,2,2,1,1,3,1,1,0,3,0,0,1,0,0,0,2,"8620 W. Emerald Street , B...",43.611373,-116.288882,Idaho,16001.0,1.600100e+14


In [13]:
df.to_csv('../data/data_geocoded.csv')

In [14]:
# print out total notebook execution time
total_seconds = int(time.time() - start_time)
minutes = total_seconds // 60
seconds = total_seconds % 60
print("--- " + str(minutes) + " minutes " + str(seconds) + " seconds ---")

--- 4 minutes 33 seconds ---
